In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from numpy.core.fromnumeric import shape
def load_data(path,batch_size,input_size):
    
    normalize = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]) 
    transform_dict = {"src":  normalize}  
    # train_path=path+"/train"
    # test_path=path+"/test"
    data = datasets.ImageFolder(root=path,transform=transform_dict["src"])
    # transform_dict1 = {"test":  normalize} 
    # test1 = datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # transform_dict = {"test":  normalize}
    # test= datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # train_size = int((1- (test_split + val_split)) * len(data))
    # test_size = int((1 - (val_split)) * len(data)) - train_size
    # val_size = len(data) - train_size - test_size
    train_size=int(0.75*len(data))
    print(len(data))
    test_size=int(len(data)-train_size)
    train, test = td.random_split(data,[train_size,test_size])

    data_loader_train = td.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    data_loader_test = td.DataLoader(test,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    # data_loader_val = td.DataLoader(val,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    return data_loader_train, data_loader_test

In [4]:
data_loader_train,data_loader_test=load_data(r"/content/drive/MyDrive/Project/datasets/Dataset2",32,40)

6922


In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 4):
        super().__init__()
#         _log_api_usage_once(self)
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes),
        )

    def forward(self, x) :
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
import torch.optim as optim
net = AlexNet(3).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [7]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
net.to(device)

num_epochs = 20
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
#         print("kjnfjnrnkrn",i)\\
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/20], Step [10/163], Loss: 1.0981, Accuracy: 40.62%
Epoch [1/20], Step [20/163], Loss: 1.0981, Accuracy: 25.00%
Epoch [1/20], Step [30/163], Loss: 1.0626, Accuracy: 50.00%
Epoch [1/20], Step [40/163], Loss: 1.0912, Accuracy: 34.38%
Epoch [1/20], Step [50/163], Loss: 1.0934, Accuracy: 37.50%
Epoch [1/20], Step [60/163], Loss: 1.1392, Accuracy: 18.75%
Epoch [1/20], Step [70/163], Loss: 1.0302, Accuracy: 31.25%
Epoch [1/20], Step [80/163], Loss: 1.1423, Accuracy: 31.25%
Epoch [1/20], Step [90/163], Loss: 1.0073, Accuracy: 50.00%
Epoch [1/20], Step [100/163], Loss: 1.1445, Accuracy: 28.12%
Epoch [1/20], Step [110/163], Loss: 1.0799, Accuracy: 40.62%
Epoch [1/20], Step [120/163], Loss: 1.1202, Accuracy: 43.75%
Epoch [1/20], Step [130/163], Loss: 1.0691, Accuracy: 40.62%
Epoch [1/20], Step [140/163], Loss: 1.0952, Accuracy: 46.88%
Epoch [1/20], Step [150/163], Loss: 1.0431, Accuracy: 62.50%
Epoch [1/20], Step [160/163], Loss: 1.1520, Accuracy: 40.62%
Epoch [2/20], Step

In [8]:
######## Write your code here #############
net.eval() 
count=0
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        # count+=1
        # print(count)
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 1731 test images: 33.10225303292894 %
